In [ ]:
import tensorflow as tf
import os
import pandas as pd
import random

In [ ]:
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            images.append(filename)
    return images

def pre_processing(data):
    label_ = []
    path_ = []
    for img in data:
        path_.append(path_to_train+img)
    for img in data:
        id_ = img.split(".")[0]
        breed = labels.loc[labels.id == id_, "breed"].values[0]
        label_.append(breed_dict[breed])
    return path_, label_

def parser(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image_resized = tf.image.resize_images(image_decoded,image_size)/255.
    return image_resized, label

In [ ]:
path_to_train = "all/train/"

data_full = load_images(path_to_train)

# Setting seed and random shuffling
random.seed(100)
random.shuffle(data_full)

# Splitting in train and dev set
train = data_full[:9982]
train.sort()
dev = data_full[9982:]
dev.sort()

# Creating labels
labels = pd.read_csv("all/labels.csv")
breeds = list(labels.breed.unique())
breed_dict = {}
for idx, breed in enumerate(breeds):
    breed_dict[breed] = idx


In [ ]:
train_path, train_label = pre_processing(train)
dev_path, dev_label = pre_processing(dev)

In [ ]:
# A vector of filenames.
filenames_train = tf.constant(train_path)
label_train = tf.one_hot(train_label,120)

# `labels[i]` is the label for the image in `filenames[i].

filenames_dev = tf.constant(dev_path)
labels_dev = tf.one_hot(dev_label, 120)

In [ ]:
image_size = (160,160)
learning_rate = 0.001
num_epochs = 2
batch_size = 32
step_size = 10
num_batches = int(len(train)/batch_size)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((filenames_train,label_train))
dataset = dataset.map(parser)
dataset = dataset.repeat(num_epochs).shuffle(buffer_size=100)
dataset = dataset.batch(batch_size)
iterator = dataset.make_one_shot_iterator()
x, y = iterator.get_next()

In [ ]:
# Construct Model

conv1 = tf.layers.conv2d(inputs=x ,filters=32, kernel_size=[3,3], padding="same", strides=1, activation=tf.nn.relu)

pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)

bnorm1 = tf.layers.batch_normalization(inputs=pool1)

conv2 = tf.layers.conv2d(inputs=bnorm1 ,filters=64, kernel_size=[3,3], padding="same", strides=1, activation=tf.nn.relu)

pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)

bnorm2 = tf.layers.batch_normalization(inputs=pool2)

conv3 = tf.layers.conv2d(inputs=bnorm2 ,filters=128, kernel_size=[3,3], padding="same", strides=1, activation=tf.nn.relu)

pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2,2], strides=2)

bnorm3 = tf.layers.batch_normalization(inputs=pool3)

flat_layer = tf.contrib.layers.flatten(bnorm3)

fc1 = tf.layers.dense(inputs=flat_layer, units=1024, activation=tf.nn.relu)

bnorm4 = tf.layers.batch_normalization(inputs=fc1)

fc2 = tf.layers.dense(inputs=bnorm4, units=512, activation=tf.nn.relu)

bnorm5 = tf.layers.batch_normalization(inputs=fc2)

logits = tf.layers.dense(inputs=bnorm5, units=120)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    for epoch in range(1,num_epochs+1):
        for step in range(1,num_batches+1):
          # Training and calculating loss for the batch
            _, batch_loss = sess.run((train_op,loss))

            #log_loss_train.append("Epoch"+str(epoch)+"Step"+str(step)+"_"+str(batch_loss))

            if (step%step_size == 0 or step==1) and (step%100 != 0):

                batch_acc = sess.run(accuracy)

                print("Epoch " + str(epoch) + "::Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(batch_loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(batch_acc))

#             if step%100 == 0 or step==num_batches:

#                 test_loss, test_acc = testing()
#                 #log_loss_test.append("Epoch"+str(epoch)+"Step"+str(step)+"_"+str(test_loss))

#                 print("Epoch " + str(epoch) + "::Step " + str(step) + ", Minibatch Loss= " + \
#                   "{:.4f}".format(batch_loss) + ", Training Accuracy= " + \
#                   "{:.3f}".format(batch_acc) + ", Testing Loss= " + \
#                   "{:.4f}".format(test_loss) + ", Testing Accuracy= " + \
#                   "{:.3f}".format(test_acc))